<a href="https://www.kaggle.com/code/typicalmango/simple-pytorch-ensemble-cnn?scriptVersionId=198579791" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np
import pandas as pd
import torch
torch.manual_seed(42)

In [2]:
device = (
    'cuda'
    if torch.cuda.is_available()
    else 'mps'
    if torch.backends.mps.is_available()
    else 'cpu'
)
print(f"Using {device}")

Using cpu


In [3]:
# This code isnt cuda compatible yet
from torch.multiprocessing import Pool, Process, set_start_method

if device=='cuda':
    try:
        set_start_method('spawn')
        print("Start method set")
    except RuntimeError:
        print("Unable to set start method")
        pass
    
device = torch.device(device)

# Loading and Preprocessing Data

In [4]:
from torchvision.transforms import transforms

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [5]:
from torchvision.datasets import FashionMNIST

train_set = FashionMNIST(root='./data', train=True, transform=train_transform, download=True)

test_set = FashionMNIST(root='./data', train=False, transform=test_transform, download=True)

100%|██████████| 26421880/26421880 [00:14<00:00, 1869624.96it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 137029.92it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 2473289.42it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 9323090.24it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [6]:
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate

train_loader = DataLoader(train_set, batch_size=64, shuffle=True, num_workers=2, persistent_workers=True, collate_fn=lambda x: [y.to(device) for y in default_collate(x)])
# train_loader = DataLoader(train_set, batch_size=64, shuffle=True, num_workers=2, collate_fn=lambda x: [y.to(device) for y in default_collate(x)])

test_loader = DataLoader(test_set, batch_size=64, num_workers=2, persistent_workers=True, collate_fn=lambda x: [y.to(device) for y in default_collate(x)])
# test_loader = DataLoader(test_set, batch_size=64, num_workers=2, collate_fn=lambda x: [y.to(device) for y in default_collate(x)])

In [7]:
labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}

# Creating the Ensemble

In [8]:
import torch.nn as nn

class net(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [9]:
net()

net(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [10]:
learning_rate = 1e-1
batch_size = 64
epochs = 5
num_models = 5

In [11]:
model_ensemble = [net().to(device) for _ in range(num_models)]

# Training the Model

In [12]:
def train_loop(data_loader, model_ensemble, loss_fn):
    size = len(data_loader.dataset)
    for i, model in enumerate(model_ensemble):
        print(f'model {i+1}:')
        optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
        model.train()
        for batch, (X, y) in enumerate(data_loader):
            pred = model(X)
            loss = loss_fn(pred, y)
            
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
            if batch % 200 == 0:
                loss, current = loss.item(), batch * batch_size + len(X)
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

        print("- - - - - - - - - - - - - - - -")
                
def test_loop(data_loader, model_ensemble, loss_fn):
    for model in model_ensemble:
        model.eval()
    size = len(data_loader.dataset)
    num_batches = len(data_loader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in data_loader:
            ensemble_preds = [model(X) for model in model_ensemble]
            pred = torch.mean(torch.stack(ensemble_preds), dim=0)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [13]:
loss_fn = nn.CrossEntropyLoss()

In [14]:
for t in range(epochs):
    print(f"Epoch {t+1}\n===============================")
    train_loop(train_loader, model_ensemble, loss_fn)
    test_loop(test_loader, model_ensemble, loss_fn)
print("Done!")

Epoch 1
model 1:
loss: 2.301896  [   64/60000]
loss: 0.678145  [12864/60000]
loss: 0.548742  [25664/60000]
loss: 0.548167  [38464/60000]
loss: 0.564799  [51264/60000]
- - - - - - - - - - - - - - - -
model 2:
loss: 2.303048  [   64/60000]
loss: 0.618793  [12864/60000]
loss: 0.315016  [25664/60000]
loss: 0.419797  [38464/60000]
loss: 0.716389  [51264/60000]
- - - - - - - - - - - - - - - -
model 3:
loss: 2.332223  [   64/60000]
loss: 0.752687  [12864/60000]
loss: 0.432703  [25664/60000]
loss: 0.463325  [38464/60000]
loss: 0.498069  [51264/60000]
- - - - - - - - - - - - - - - -
model 4:
loss: 2.329745  [   64/60000]
loss: 0.416089  [12864/60000]
loss: 0.556852  [25664/60000]
loss: 0.509201  [38464/60000]
loss: 0.602536  [51264/60000]
- - - - - - - - - - - - - - - -
model 5:
loss: 2.296519  [   64/60000]
loss: 0.788691  [12864/60000]
loss: 0.545077  [25664/60000]
loss: 0.578153  [38464/60000]
loss: 0.544312  [51264/60000]
- - - - - - - - - - - - - - - -
Test Error: 
 Accuracy: 83.1%, Avg lo

# Evaluation

In [15]:
def test_compare(data_loader, model_ensemble, loss_fn):
    for model in model_ensemble:
        model.eval()
    size = len(data_loader.dataset)
    num_batches = len(data_loader)
    
    with torch.no_grad():
        # Comoaring Individual Models
        individual_loss = []
        individual_correct = []
        
        for model in model_ensemble:
            test_loss, correct = 0, 0
            
            for X, y in data_loader:
                pred = model(X)
                test_loss += loss_fn(pred, y)
                correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
            test_loss /= num_batches
            correct /= size
            individual_loss.append(test_loss)
            individual_correct.append(correct)
        print("Individual Performance: ")
        for i in range(num_models):
            print(f"Model {i+1}: \n Accuracy: {(100*individual_correct[i]):>0.1f}%, Avg loss: {individual_loss[i]:>8f}")

        print("\n===============================")
        
        print(f"\nAverage individual performance: \n Accuracy: {(100*sum(individual_correct)/num_models):>0.1f}%, Avg loss: {sum(individual_loss)/num_models:>8f} \n")
        
        print("===============================")
        
        test_loss, correct = 0, 0
        for X, y in data_loader:
            ensemble_preds = [model(X) for model in model_ensemble]
            pred = torch.mean(torch.stack(ensemble_preds), dim=0)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Ensemble Performance: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [16]:
test_compare(test_loader, model_ensemble, loss_fn)

Individual Performance: 
Model 1: 
 Accuracy: 85.7%, Avg loss: 0.388577
Model 2: 
 Accuracy: 83.1%, Avg loss: 0.468489
Model 3: 
 Accuracy: 86.6%, Avg loss: 0.373466
Model 4: 
 Accuracy: 86.7%, Avg loss: 0.360495
Model 5: 
 Accuracy: 85.2%, Avg loss: 0.424311


Average individual performance: 
 Accuracy: 85.5%, Avg loss: 0.403067 

Ensemble Performance: 
 Accuracy: 87.6%, Avg loss: 0.346719 



# Conclusion
Therefore, it can be concluded that the ensemble model outperforms the individual models. <br>
The whole is greater than the sum of its parts!